<a href="https://colab.research.google.com/github/Akshitha-Maroju/-Audio-Steganography-with-GUI-in-Google-Colab-/blob/main/Stegnography2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scipy numpy


In [2]:
from google.colab import files
uploaded = files.upload()  # Upload your uncompressed .wav file

# Set your uploaded file name
audio_file = "file_example_WAV_2MG.wav"  # Replace with exact file name if different


Saving file_example_WAV_2MG.wav to file_example_WAV_2MG.wav


In [3]:
# ✅ STEP 1: Upload your .wav file
from google.colab import files
uploaded = files.upload()  # Choose your file: file_example_WAV_2MG.wav

# ✅ STEP 2: Define the uploaded file name
audio_file = list(uploaded.keys())[0]  # Automatically get the uploaded file name

# ✅ STEP 3: Import libraries
import numpy as np
from scipy.io import wavfile

# ✅ STEP 4: Utility to convert text to bits
def text_to_bits(text):
    return ''.join(format(ord(c), '08b') for c in text) + '1111111111111110'  # End marker

# ✅ STEP 5: Encode function to hide message in audio
def encode_audio(wav_path, secret_message, output_path="stego_audio.wav"):
    rate, data = wavfile.read(wav_path)
    bits = text_to_bits(secret_message)

    # Use only one channel if stereo
    if data.ndim > 1:
        data = data[:, 0]

    flat_data = data.flatten()
    if len(bits) > len(flat_data):
        raise ValueError("Message too long to hide in this audio!")

    encoded = flat_data.copy()
    for i in range(len(bits)):
        encoded[i] = (encoded[i] & ~1) | int(bits[i])

    wavfile.write(output_path, rate, encoded)
    print("✅ Message encoded and saved as:", output_path)

# ✅ STEP 6: Decode function to extract message
def bits_to_text(bits):
    chars = [bits[i:i+8] for i in range(0, len(bits), 8)]
    message = ''
    for b in chars:
        if b == '11111110':  # End marker
            break
        message += chr(int(b, 2))
    return message

def decode_audio(wav_path):
    rate, data = wavfile.read(wav_path)
    if data.ndim > 1:
        data = data[:, 0]
    flat_data = data.flatten()
    bits = ''.join([str(sample & 1) for sample in flat_data])
    return bits_to_text(bits)

# ✅ STEP 7: Hide your secret message
secret = "This is a hidden secret in your audio file!"
encode_audio(audio_file, secret)

# ✅ STEP 8: Extract and display the hidden message
decoded = decode_audio("stego_audio.wav")
print("🔓 Hidden Message:", decoded)

# ✅ STEP 9: Optionally download the output file
files.download("stego_audio.wav")


Saving file_example_WAV_2MG.wav to file_example_WAV_2MG (1).wav
✅ Message encoded and saved as: stego_audio.wav
🔓 Hidden Message: This is a hidden secret in your audio file!ÿ


/tmp/ipython-input-3-1554681192.py:18: WavFileWarning: Chunk (non-data) not understood, skipping it.
  rate, data = wavfile.read(wav_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [4]:
!pip install gradio scipy numpy


In [9]:
!pip install gradio scipy --quiet

import gradio as gr
import numpy as np
from scipy.io import wavfile
import tempfile

# -- Text conversion functions --
def text_to_bits(text):
    return ''.join(format(ord(c), '08b') for c in text) + '11111110'

def bits_to_text(bits):
    chars = [bits[i:i+8] for i in range(0, len(bits), 8)]
    message = ''
    for b in chars:
        if b == '11111110':
            break
        message += chr(int(b, 2))
    return message

# -- Encode --
def encode_audio(audio_file, message):
    print(">>> Encoding...")
    print("Audio File:", audio_file)
    print("Message:", message)

    if audio_file is None or message.strip() == "":
        return "❌ Error: Please provide both audio and message.", None

    try:
        rate, data = wavfile.read(audio_file)
        print("Audio Rate:", rate)
        print("Data Shape:", data.shape)

        if data.ndim > 1:
            data = data[:, 0]

        bits = text_to_bits(message)
        flat_data = data.flatten()

        if len(bits) > len(flat_data):
            return "❌ Message too long to hide in this audio.", None

        encoded = flat_data.copy()
        for i in range(len(bits)):
            encoded[i] = (encoded[i] & ~1) | int(bits[i])

        temp = tempfile.NamedTemporaryFile(delete=False, suffix=".wav")
        wavfile.write(temp.name, rate, encoded.astype(data.dtype))

        print("✅ Stego audio created:", temp.name)
        return "✅ Message encoded successfully!", temp.name

    except Exception as e:
        print("❌ Error during encoding:", e)
        return f"❌ Error: {str(e)}", None

# -- Decode --
def decode_audio(stego_file):
    print(">>> Decoding...")
    if stego_file is None:
        return "❌ Please upload a .wav file."

    try:
        rate, data = wavfile.read(stego_file)
        if data.ndim > 1:
            data = data[:, 0]

        bits = ''.join(str(sample & 1) for sample in data.flatten())
        message = bits_to_text(bits)
        print("✅ Message decoded:", message)
        return message
    except Exception as e:
        print("❌ Error during decoding:", e)
        return f"❌ Error: {str(e)}"

# -- Gradio UI --
with gr.Blocks() as demo:
    gr.Markdown("## 🔐 EchoCrypt: Hide & Reveal Messages in Audio")

    with gr.Tab("Encode"):
        with gr.Row():
            audio_in = gr.Audio(type="filepath", label="Upload Clean Audio (.wav)")
            text_in = gr.Textbox(lines=2, placeholder="Enter your secret message")
        encode_btn = gr.Button("Encode Message")
        encode_status = gr.Textbox(label="Status")
        encoded_file = gr.File(label="Download Stego Audio")

        encode_btn.click(encode_audio, inputs=[audio_in, text_in], outputs=[encode_status, encoded_file])

    with gr.Tab("Decode"):
        stego_in = gr.Audio(type="filepath", label="Upload Stego Audio (.wav)")
        decode_btn = gr.Button("Decode Message")
        decoded_text = gr.Textbox(label="Hidden Message")

        decode_btn.click(decode_audio, inputs=stego_in, outputs=decoded_text)

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://586c1cd90e7761c419.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
